In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ClimateWatch ETL") \
    .getOrCreate()

In [2]:
GDELT_PATH = "/home/jovyan/work/data_storage/gdelt/*.csv"
CLIMATE_PATH = "/home/jovyan/work/data_storage/climate/*.csv"

In [3]:
gdelt_df = spark.read.option("header", False).csv("/home/jovyan/work/data_storage/gdelt/*.csv")
climate_df = spark.read.option("header", True).csv("/home/jovyan/work/data_storage/climate/*.csv")

In [4]:
gdelt_columns = [
    "GLOBALEVENTID", "SQLDATE", "MonthYear", "Year", "FractionDate", "Actor1Code", "Actor1Name", "Actor1CountryCode",
    "Actor1KnownGroupCode", "Actor1EthnicCode", "Actor1Religion1Code", "Actor1Religion2Code", "Actor1Type1Code",
    "Actor1Type2Code", "Actor1Type3Code", "Actor2Code", "Actor2Name", "Actor2CountryCode", "Actor2KnownGroupCode",
    "Actor2EthnicCode", "Actor2Religion1Code", "Actor2Religion2Code", "Actor2Type1Code", "Actor2Type2Code",
    "Actor2Type3Code", "IsRootEvent", "EventCode", "EventBaseCode", "EventRootCode", "QuadClass", "GoldsteinScale",
    "NumMentions", "NumSources", "NumArticles", "AvgTone", "Actor1Geo_Type", "Actor1Geo_FullName", "Actor1Geo_CountryCode",
    "Actor1Geo_ADM1Code", "Actor1Geo_Lat", "Actor1Geo_Long", "Actor1Geo_FeatureID", "Actor2Geo_Type", "Actor2Geo_FullName",
    "Actor2Geo_CountryCode", "Actor2Geo_ADM1Code", "Actor2Geo_Lat", "Actor2Geo_Long", "Actor2Geo_FeatureID", "ActionGeo_Type",
    "ActionGeo_FullName", "ActionGeo_CountryCode", "ActionGeo_ADM1Code", "ActionGeo_Lat", "ActionGeo_Long", "ActionGeo_FeatureID",
    "DATEADDED", "SOURCEURL"
]
gdelt_df = gdelt_df.toDF(*gdelt_columns)
gdelt_df.select("SQLDATE", "SOURCEURL").show(5)

+--------+--------------------+
| SQLDATE|           SOURCEURL|
+--------+--------------------+
|       1|                  57|
|20230508|https://www.euras...|
|20230508|https://www.euras...|
|20230508|https://www.livem...|
|20230508|https://www.nsnew...|
+--------+--------------------+
only showing top 5 rows



In [5]:
from pyspark.sql.functions import col

gdelt_df = gdelt_df.withColumn("year", col("SQLDATE").substr(1, 4))

In [6]:
joined = gdelt_df.join(climate_df, gdelt_df.year == climate_df.year, how="left")
print("Birleşik satır sayısı:", joined.count())
joined.show(5)

Birleşik satır sayısı: 1536604
+-------------+--------+---------+----+------------+----------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+----------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+-----------+---------+-------------+-------------+---------+--------------+-----------+----------+-----------+-----------------+--------------+--------------------+---------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+-------------+--------------+-------------------+---------+--------------------+------------+------------+----+-----+
|GLOBALEVE